In [1]:
import numpy as np
import pandas as pd
from statistics import median

import math

from sklearn.preprocessing import MinMaxScaler

In [15]:
train_data = pd.read_csv('../train.data',delimiter=',')
matrix = np.zeros((943,1682),dtype=int)
for row in train_data.to_numpy():
    matrix[row[0]-1,row[1]-1] = row[2]
test = pd.read_csv('../test.data').to_numpy()
users = test.transpose()[0,:10]
users

In [3]:
Rmin = 1
Rmax = 5
Rmid = int((5+1)/2)

def med(arr1,arr2):
    midup = []
    middown = []
    for item in range(len(arr1)):
        if arr1[item]<Rmid:
            middown.append(arr1[item])
        if arr1[item] >= Rmid:
            midup.append(arr1[item])
        if arr2[item] < Rmid:
            middown.append(arr2[item])
        if  arr2[item] >= Rmid:
            midup.append(arr2[item])
    return median(midup),median(middown)

def proximity(r1,r2,midplus,midminus):
    if (r1>=Rmid and r2>=Rmid) or (r1<Rmid and r2<Rmid):
        prox = (((abs(r1-r2))-(midminus+midplus)/2)/(Rmax-Rmin))**2
    else:
        dst = abs(r1-r2)
        if dst > Rmid:
            delta = 0.75
        elif dst == Rmid:
            delta = 0.5
        else:
            delta = 0.25
        prox = delta*(((Rmax-Rmin)/delta)**2)
    return prox

def impact(r1,r2,midplus,midminus):
    if (r1>=Rmid and r2>=Rmid) or (r1<Rmid and r2<Rmid):
        imp = math.pow(math.e,-(1/(abs(r1-Rmid)+1)*(abs(r2-Rmid)+1)))
    else:
        imp = 1/(abs(r1-Rmid)+1)*(abs(r2-Rmid)+1)
    return imp

def popularity(r1,r2,col_no):
    avg = matrix.mean(axis=0)
    if (r1>=Rmid and r2>=Rmid) or (r1<Rmid and r2<Rmid):
        pop = math.log10((2+(((r1+r2)/2)-avg[col_no])**2))
    else:
        pop = 0.3010
    return pop

def MPIP(u1,u2,item):
    midplus,midminus = med(matrix[u1],matrix[u2])
    prox = proximity(matrix[u1,item],matrix[u2,item],midplus,midminus)
    imp = impact(matrix[u1,item],matrix[u2,item],midplus,midminus)
    pop = popularity(matrix[u1,item],matrix[u2,item],item)
    return prox*imp*pop

def sim(u1,u2):
    sum=0
    for i in range(0,1678,4):
        t1 =MPIP(u1,u2,i)
        t2 =MPIP(u1,u2,i+1)
        t3 =MPIP(u1,u2,i+2)
        t4 =MPIP(u1,u2,i+3)
        sum+=(t1+t2+t3+t4)

    return sum

In [4]:
Mpip = np.zeros((943,943))
for i in range(943):
    if i in users1:
        for j in range(943):
            if i==j:
                Mpip[i,j]=654.298
                continue
            Mpip[i,j] = sim(i,j)

In [5]:
with open('../mpip1.csv','w') as f:
        for i in Mpip:
            count=0
            for j in i:
                f.write(str(j))
                if count<942:
                    f.write(',')
                count+=1
            f.write('\n')


In [17]:
Mpip1 = pd.read_csv('../mpip1.csv',delimiter=',',names=[str(i) for i in range(1,944)]).to_numpy()

In [19]:
users = users[:5]
Mpip1[users[:]]

array([[2857.0284282 , 3173.6325693 , 3209.50166453, ..., 3135.4490432 ,
        3151.54425851, 3045.64175663],
       [1210.37206686, 1004.35704824, 1061.21158753, ...,  855.43893043,
        1206.58026242, 1066.67613979],
       [ 915.65264204,  587.22828393,  513.0713679 , ...,  504.68175252,
         630.62648582,  733.44232586],
       [ 983.09238658,  386.86245001,  425.87105445, ...,  416.13026514,
         605.73447256,  708.44656   ],
       [1948.22919879, 2347.36469034, 2383.65173565, ..., 2375.12075908,
        2284.31833597, 2263.61811126]])

In [20]:
users

array([804, 524, 207, 807,   5], dtype=int64)

In [21]:
mpip = np.zeros((943,943))

In [23]:
for i in users:
    mpip[i-1] = Mpip1[i]

In [28]:
users = test.transpose()[0,:10]
users = users[5:10]

In [ ]:
for i in users:
    mpip[i-1] = Mpip1[i]

In [27]:
df = pd.DataFrame(mpip)
df.to_csv('../mpip.csv',sep=',',index=False)